In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


In [59]:
with Path("../13.in").open() as f:
    data = f.read().split("\n\n")
    data = [d.split("\n") for d in data]


In [64]:
# "." = ash
# "#" = rocks

testdata = """\
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#""".split("\n\n")
testdata = [d.split("\n") for d in testdata]
testdata

[['#.##..##.',
  '..#.##.#.',
  '##......#',
  '##......#',
  '..#.##.#.',
  '..##..##.',
  '#.#.##.#.'],
 ['#...##..#',
  '#....#..#',
  '..##..###',
  '#####.##.',
  '#####.##.',
  '..##..###',
  '#....#..#']]

## Part I - Find the reflection axis

In [137]:
np.array([
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1]
]).T

array([[0, 0, 1, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [170]:
def axis_to_points(axis, val):
    if axis == "y":
        cols_left_of_line = val + 1
        return cols_left_of_line * 100
    elif axis == "x":
        rows_above_line = val + 1
        return rows_above_line


def find_mirror_axis(pattern):
    pattern = np.array([list(row) for row in pattern if row])
    # print(pattern)

    for pat, axis in zip([pattern, pattern.T], ["y", "x"]):
        for pos in range(0, pat.shape[0]-1):
            matches = pat[pos, :] == pat[pos+1, :]

            if not all(matches):
                continue

            # print(f"Possible match between {axis}={pos} and {axis}={pos+1}")
            pos_a, pos_b = pos-1, pos+2
            while pos_a >= 0 and pos_b < pat.shape[0]:
                # print(f"Checking {axis} {pos_a}<->{pos_b}")
                matches = pat[pos_a, :] == pat[pos_b, :]
                if not all(matches):
                    # print(f"Match broken at {pos_a}<->{pos_b} -> {matches}")
                    break
                pos_a -= 1
                pos_b += 1
            else:
                # All possible reflections checked
                return axis, pos

    print("No match found! -> 0 pts")
    return 0


def solve1(data):
    points = 0
    for pattern in data:
        axis, pos = find_mirror_axis(pattern)
        score = axis_to_points(axis, pos)
        # print(f"Match found between {axis}={pos} and {axis}={pos+1} -> {score} pts")  # 1-indexed
        points += score
    return points

assert (a := solve1(testdata)) == 405, a

In [171]:
solve1(data)

35232

## Part II - Smudged mirror

- every mirror has **exactly one smudge**.

- Locate and fix the smudge that causes a **different reflection line** to be valid.

In [177]:
def find_off_by_ones(pattern):
    candidates = []

    for pat, axis in zip([pattern, pattern.T], ["y", "x"]):
        for pos in range(0, pat.shape[0]-1):
            matches = pat[pos, :] == pat[pos+1, :]

            if np.count_nonzero(~matches) == 1:
                for offset in [0, 1]:
                    if axis == "y":
                        y = pos + offset
                        x = np.where(~matches)[0][0]
                    else:
                        x = pos + offset
                        y = np.where(~matches)[0][0]
                    candidates.append((y, x))

            if not all(matches):
                continue

            # print(f"Possible match between {axis}={pos} and {axis}={pos+1}")
            pos_a, pos_b = pos-1, pos+2
            while pos_a >= 0 and pos_b < pat.shape[0]:
                # print(f"Checking {axis} {pos_a}<->{pos_b}")
                matches = pat[pos_a, :] == pat[pos_b, :]

                if np.count_nonzero(~matches) == 1:
                    for p in [pos_a, pos_b]:
                        if axis == "y":
                            y = p
                            x = np.where(~matches)[0][0]
                        else:
                            x = p
                            y = np.where(~matches)[0][0]
                        candidates.append((y, x))

                if not all(matches):
                    # print(f"Match broken at {pos_a}<->{pos_b} -> {matches}")
                    break
                pos_a -= 1
                pos_b += 1

    return candidates


def find_smudged_mirror_axis(base_pattern):
    previous_solution = find_mirror_axis(base_pattern)

    base_pattern = np.array([list(row) for row in base_pattern if row])

    flip = {"#": ".", ".": "#"}

    candidates = find_off_by_ones(base_pattern)
    # print(candidates)

    for cand in candidates:
        pattern = base_pattern.copy()
        pattern[cand] = flip[pattern[cand]]

        for pat, axis in zip([pattern, pattern.T], ["y", "x"]):
            for pos in range(0, pat.shape[0]-1):
                matches = pat[pos, :] == pat[pos+1, :]

                if not all(matches):
                    continue

                # print(f"Possible match between {axis}={pos} and {axis}={pos+1}")
                pos_a, pos_b = pos-1, pos+2
                while pos_a >= 0 and pos_b < pat.shape[0]:
                    # print(f"Checking {axis} {pos_a}<->{pos_b}")
                    matches = pat[pos_a, :] == pat[pos_b, :]
                    if not all(matches):
                        # print(f"Match broken at {pos_a}<->{pos_b} -> {matches}")
                        break
                    pos_a -= 1
                    pos_b += 1
                else:
                    # All possible reflections checked
                    if (axis, pos) != previous_solution:
                        # This solution is new
                        return axis, pos

    print("No match found! -> 0 pts")
    return 0


def solve2(data):
    points = 0
    for pattern in data:

        axis, pos = find_smudged_mirror_axis(pattern)
        score = axis_to_points(axis, pos)
        print(f"Match found between {axis}={pos} and {axis}={pos+1} -> {score} pts")  # 1-indexed
        points += score
    return points

assert (a := solve2(testdata)) == 400, a

Match found between y=2 and y=3 -> 300 pts
Match found between y=0 and y=1 -> 100 pts


In [178]:
solve2(data)

Match found between y=10 and y=11 -> 1100 pts
Match found between y=2 and y=3 -> 300 pts
Match found between y=1 and y=2 -> 200 pts
Match found between y=9 and y=10 -> 1000 pts
Match found between y=1 and y=2 -> 200 pts
Match found between y=11 and y=12 -> 1200 pts
Match found between x=0 and x=1 -> 1 pts
Match found between y=5 and y=6 -> 600 pts
Match found between y=9 and y=10 -> 1000 pts
Match found between y=2 and y=3 -> 300 pts
Match found between y=0 and y=1 -> 100 pts
Match found between y=2 and y=3 -> 300 pts
Match found between x=10 and x=11 -> 11 pts
Match found between y=4 and y=5 -> 500 pts
Match found between y=3 and y=4 -> 400 pts
Match found between x=4 and x=5 -> 5 pts
Match found between x=13 and x=14 -> 14 pts
Match found between x=10 and x=11 -> 11 pts
Match found between x=0 and x=1 -> 1 pts
Match found between x=6 and x=7 -> 7 pts
Match found between y=12 and y=13 -> 1300 pts
Match found between y=6 and y=7 -> 700 pts
Match found between y=3 and y=4 -> 400 pts
Mat

37982